In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习：MLOps阶段4：形式化：使用Google Artifact Registry入门

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage4/get_started_with_google_artifact_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage4/get_started_with_google_artifact_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage4/get_started_with_google_artifact_registry.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                               
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端的MLOps生产。本教程涵盖了第4阶段：规范化：开始使用Google Artifact Registry。

### 目标

在本教程中，您将学习如何使用`Google Artifact Registry`。

本教程使用以下Google Cloud ML服务：

- `Google Artifact Registry`

执行的步骤包括：

- 创建私有的Docker仓库。
- 为私有的Docker仓库标记一个容器镜像。
- 将容器镜像推送到私有的Docker仓库。
- 从私有的Docker仓库拉取一个容器镜像。
- 删除私有的Docker仓库。

数据集

本教程中不使用任何数据集。对示例数据集的引用仅用于演示目的。

### 成本

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

安装

安装执行笔记本所需的软件包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q

重新启动内核

一旦您安装了额外的包，您需要重新启动笔记本内核，这样它才能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*确保如果有此选项，则在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，都需要执行以下步骤。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，将获得 $300 的免费信用用于计算/存储成本。

2. [确保您的项目已启用结算。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、计算引擎 API 和云存储。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，则需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的命令作为 shell 命令，并且插值以 `$` 为前缀的 Python 变量。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分操作。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行现场教程会话，您可能会使用共享的测试账户或项目。为了避免在创建资源时用户之间发生名称冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您使用的是Vertex AI Workbench笔记本**，则您的环境已经通过验证。请跳过此步骤。

**如果您使用的是Colab**，请运行下面的单元格，并按照提示进行身份验证，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**单击创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，然后单击**创建**。

在**将此服务帐号授予项目访问权限**部分，单击角色下拉列表。在过滤框中键入"Vertex"，并选择**Vertex管理员**。在过滤框中键入"Storage Object Admin"，并选择**存储对象管理员**。

单击创建。包含您的密钥的JSON文件将下载到本地环境中。

在下面的单元格中将您的服务帐号密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

## Google Artifact Registry简介

`Google Artifact Registry`是一个用于存储和管理私有存储库中的构件的服务，包括容器镜像、Helm图表和语言包。它是Google Cloud推荐的容器镜像注册表。

了解更多关于[Docker快速入门](https://cloud.google.com/artifact-registry/docs/docker/quickstart)。

### 启用 Artifact Registry API

首先，您必须为您的项目启用Artifact Registry API服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

## 创建一个私有的Docker仓库

你的第一步是在Google Artifact Registry中创建自己的Docker仓库。

1. 运行`gcloud artifacts repositories create`命令来创建一个新的Docker仓库，指定所在地区并填写描述为"docket仓库"。

2. 运行`gcloud artifacts repositories list`命令来验证你的仓库是否已创建成功。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

### 配置私有仓库的身份验证

在推送或拉取容器镜像之前，配置Docker使用`gcloud`命令行工具来对区域内的`Artifact Registry`发出身份验证请求。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

获取一个示例容器镜像

为了演示目的，您可以获取（拉取）我们示例容器镜像`hello-app:1.0`的本地副本。

In [ ]:
! docker pull us-docker.pkg.dev/google-samples/containers/gke/hello-app:1.0

给您的容器镜像打标签

现在您有了自己的容器镜像，第一步就是给您的镜像打标签。

- 使用仓库名称给 Docker 镜像打标签会配置 docker push 命令将镜像推送到特定位置，例如，us-central1-docker.pkg.dev。

- `:my-tag` 是您添加到 Docker 镜像的标签。如果未指定标签，则默认为 `:latest`。

In [ ]:
CONTAINER_NAME = "my-image:my-tag"

! docker tag us-docker.pkg.dev/google-samples/containers/gke/hello-app:1.0 us-central1-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{CONTAINER_NAME}

将您的镜像推送到您的私有Docker仓库

接下来，将您的容器推送到您的私有Docker仓库。

In [ ]:
! docker push {REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{CONTAINER_NAME}

现在从您的私人Docker存储库中拉取您的容器。

In [ ]:
! docker pull {REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{CONTAINER_NAME}

### 删除您的私有 Docker 仓库

最后，一旦您的私有仓库变得过时，使用命令 `gcloud artifacts repositories delete` 在 `Google Artifact Registry` 中将其删除。

In [ ]:
! gcloud artifacts repositories delete {PRIVATE_REPO} --location={REGION} --quiet